# Pydantic Validation

## What You'll Learn
- Using field validators to validate individual fields
- Creating model validators for cross-field validation
- Built-in field types (EmailStr, HttpUrl, PositiveInt, etc.)
- Writing custom validators
- Validation order and dependencies
- Handling and customizing validation errors

---

## The Problem: Basic Type Checking Isn't Enough

Pydantic validates types automatically, but sometimes you need more sophisticated rules:

In [ ]:
from pydantic import BaseModel

# ❌ Type validation alone isn't sufficient
class User(BaseModel):
    username: str
    age: int

# These are valid types but invalid data!
user1 = User(username="", age=-5)  # Empty username, negative age
user2 = User(username="ab", age=200)  # Too short username, unrealistic age

print(user1)
print(user2)

**Problems:**
- Username can be empty or too short
- Age can be negative or unrealistic
- No business logic validation
- Need custom validation rules

---

## Field Validators

### Using field_validator Decorator

Field validators check individual fields and can transform or reject values:

In [ ]:
from pydantic import BaseModel, field_validator, ValidationError

class User(BaseModel):
    """User with validated fields."""
    username: str
    age: int
    
    @field_validator('username')
    @classmethod
    def validate_username(cls, value: str) -> str:
        """Validate username is not empty and at least 3 characters."""
        if not value:
            raise ValueError('Username cannot be empty')
        if len(value) < 3:
            raise ValueError('Username must be at least 3 characters')
        return value
    
    @field_validator('age')
    @classmethod
    def validate_age(cls, value: int) -> int:
        """Validate age is between 0 and 120."""
        if value < 0:
            raise ValueError('Age cannot be negative')
        if value > 120:
            raise ValueError('Age must be 120 or less')
        return value

# Valid user
user = User(username="alice", age=25)
print(user)

In [ ]:
# Invalid username (too short)
try:
    user = User(username="ab", age=25)
except ValidationError as e:
    print(e)

In [ ]:
# Invalid age (negative)
try:
    user = User(username="alice", age=-5)
except ValidationError as e:
    print(e)

**What happens:**
1. `@field_validator('username')` decorates a method to validate `username` field
2. Validator receives the field value, checks it, and returns validated value
3. If validation fails, raises `ValueError` with error message
4. Validators run automatically when creating instances

### Transforming Values in Validators

In [ ]:
from pydantic import BaseModel, field_validator

class Product(BaseModel):
    """Product with normalized name."""
    name: str
    price: float
    
    @field_validator('name')
    @classmethod
    def normalize_name(cls, value: str) -> str:
        """Convert name to title case and strip whitespace."""
        return value.strip().title()
    
    @field_validator('price')
    @classmethod
    def validate_price(cls, value: float) -> float:
        """Ensure price is positive and round to 2 decimals."""
        if value <= 0:
            raise ValueError('Price must be positive')
        return round(value, 2)

# Input has inconsistent formatting
product = Product(name="  laptop computer  ", price=999.9999)
print(product)
print(f"Name: '{product.name}'")
print(f"Price: {product.price}")

---

## Model Validators (Cross-Field Validation)

Use model validators when validation depends on multiple fields:

In [ ]:
from pydantic import BaseModel, model_validator, ValidationError
from typing import Optional

class PasswordReset(BaseModel):
    """Password reset form."""
    password: str
    confirm_password: str
    
    @model_validator(mode='after')
    def validate_passwords_match(self):
        """Ensure password and confirm_password match."""
        if self.password != self.confirm_password:
            raise ValueError('Passwords do not match')
        return self

# Valid: passwords match
reset1 = PasswordReset(password="SecureP@ss123", confirm_password="SecureP@ss123")
print(reset1)

In [ ]:
# Invalid: passwords don't match
try:
    reset2 = PasswordReset(password="SecureP@ss123", confirm_password="DifferentPass")
except ValidationError as e:
    print(e)

### Validating Date Ranges

In [ ]:
from pydantic import BaseModel, model_validator, ValidationError
from datetime import date

class DateRange(BaseModel):
    """Date range with validation."""
    start_date: date
    end_date: date
    
    @model_validator(mode='after')
    def validate_date_range(self):
        """Ensure end_date is after start_date."""
        if self.end_date < self.start_date:
            raise ValueError('end_date must be after start_date')
        return self

# Valid date range
range1 = DateRange(start_date="2024-01-01", end_date="2024-12-31")
print(range1)

In [ ]:
# Invalid: end before start
try:
    range2 = DateRange(start_date="2024-12-31", end_date="2024-01-01")
except ValidationError as e:
    print(e)

---

## Built-in Field Types

Pydantic provides specialized types with built-in validation:

In [ ]:
from pydantic import BaseModel, EmailStr, HttpUrl, PositiveInt, constr, ValidationError

class UserProfile(BaseModel):
    """User profile with specialized types."""
    email: EmailStr  # Validates email format
    website: HttpUrl  # Validates URL format
    age: PositiveInt  # Must be positive integer
    username: constr(min_length=3, max_length=20)  # String with length constraints

# Valid profile
profile = UserProfile(
    email="alice@example.com",
    website="https://alice.dev",
    age=28,
    username="alice123"
)
print(profile)

In [ ]:
# Invalid email
try:
    profile = UserProfile(
        email="not-an-email",
        website="https://example.com",
        age=25,
        username="bob"
    )
except ValidationError as e:
    print(e)

### Common Built-in Types

| Type | Validates |
|------|----------|
| `EmailStr` | Valid email format |
| `HttpUrl` | Valid HTTP/HTTPS URL |
| `PositiveInt` | Integer > 0 |
| `PositiveFloat` | Float > 0 |
| `NegativeInt` | Integer < 0 |
| `constr(...)` | String with constraints (length, pattern) |
| `conint(...)` | Integer with constraints (range) |
| `confloat(...)` | Float with constraints (range) |

### Using Constraints

In [ ]:
from pydantic import BaseModel, conint, confloat, constr

class Product(BaseModel):
    """Product with constrained fields."""
    name: constr(min_length=1, max_length=100)
    quantity: conint(ge=0, le=1000)  # ge = greater or equal, le = less or equal
    price: confloat(gt=0, le=1_000_000)  # gt = greater than
    discount: confloat(ge=0, le=1)  # Between 0 and 1 (0% to 100%)

product = Product(
    name="Laptop",
    quantity=50,
    price=999.99,
    discount=0.15  # 15% discount
)
print(product)

---

## Best Practices

### ✅ Do:
- Use field validators for single-field validation
- Use model validators for cross-field validation
- Use built-in types (EmailStr, PositiveInt) when available
- Provide clear, specific error messages
- Return the validated/transformed value from validators
- Use `@classmethod` decorator for field validators

### ❌ Don't:
- Don't perform complex business logic in validators
- Don't raise generic exceptions (use ValueError)
- Don't modify other fields in a field validator
- Don't forget to return the value from validators
- Don't use validators when simple type constraints suffice

### Examples:

In [ ]:
from pydantic import BaseModel, field_validator, PositiveInt

# ✅ Good: Clear validation with specific error
class User(BaseModel):
    username: str
    age: PositiveInt  # Use built-in type
    
    @field_validator('username')
    @classmethod
    def validate_username(cls, value: str) -> str:
        """Validate username format."""
        if len(value) < 3:
            raise ValueError('Username must be at least 3 characters')
        if not value.isalnum():
            raise ValueError('Username must be alphanumeric')
        return value.lower()  # Transform to lowercase

user = User(username="Alice123", age=25)
print(user)
print(f"Username is lowercase: {user.username}")

---

## Summary

### Key Concepts:
- **Field validators** validate and transform individual fields
- **Model validators** validate across multiple fields
- Use `@field_validator('field_name')` for field validation
- Use `@model_validator(mode='after')` for cross-field validation
- Built-in types like `EmailStr`, `PositiveInt` provide automatic validation
- Validators must return the validated value
- Raise `ValueError` with descriptive messages for validation failures

### Syntax Reference:
```python
from pydantic import BaseModel, field_validator, model_validator

class MyModel(BaseModel):
    field1: str
    field2: int
    
    # Field validator
    @field_validator('field1')
    @classmethod
    def validate_field1(cls, value: str) -> str:
        if not value:
            raise ValueError('Cannot be empty')
        return value
    
    # Model validator (cross-field)
    @model_validator(mode='after')
    def validate_model(self):
        if self.field2 < 0:
            raise ValueError('field2 must be positive')
        return self
```

### Next Steps:
Next, learn about [Pydantic Advanced Features](05-pydantic-advanced.ipynb) including nested models, configuration options, and real-world patterns.